If you any found any errors, please follow [unsloth ORPO](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-ORPO.ipynb)

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name = "USER/SFT_MODEL",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Unsloth 2024.12.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
PAD_TOKEN = "<|pad|>"

In [ ]:
import pandas as pd

df_train = pd.read_excel("/content/training_sft_multilingual_data.xlsx")
df_valid = pd.read_excel("/content/validation_sft_multilingual_data.xlsx")

In [ ]:
df_valid.head(2)

,Unnamed: 0,Original_Language,Original_Title,English_Title,Poet,Translator,Original_Poem,English_Poem,token_length,formatted_text
0,0,HIndi,Ghar Ki Yaad,Memory of Home,Kumar Vishwas\n,NaN,"गुलाब की खुशबू में बसी है,\nहरियाली में ढलती स...","In the fragrance of roses, it resides, \nIn t...",232,<|begin_of_text|><|start_header_id|>system<|en...
1,1,Assamese,Nadi,The River,Bhupen Hazarika,Various,"নদী\nনদী, তুঁতৰ বাণী বহে\nপ্ৰবাহিত শান্তিৰ বাট...","The River\nO River, your voice flows\nAlong th...",281,<|begin_of_text|><|start_header_id|>system<|en...


In [ ]:
df_train.columns , df_valid.columns

(Index(['Unnamed: 0', 'Original_Language', 'Original_Title', 'English_Title',
        'Original_Poem', 'English_Poem', 'Poet', 'Translator', 'token_length',
        'formatted_text', 'trained_trans', 'untrianed_translation'],
       dtype='object'),
 Index(['Unnamed: 0', 'Original_Language', 'Original_Title', 'English_Title',
        'Poet', 'Translator', 'Original_Poem', 'English_Poem', 'token_length',
        'formatted_text'],
       dtype='object'))

In [ ]:
import pandas as pd


df_multi = pd.read_excel("/content/multi_poems_trained+untrained.xlsx")

# Perform the inner join
merged_df = pd.merge(df_train, df_multi, on="English_Poem", how="inner")
merged_df_valid = pd.merge(df_valid, df_multi, on="English_Poem", how="inner")

merged_df_valid = merged_df_valid[['Original_Language_x' , 'Original_Poem_x', 'English_Poem' , 'untrianed_translation', 'trained_trans' ]]
merged_df_valid.columns = ['Original_Language' , 'Original_Poem' , 'English_Poem' , 'untrianed_translation' , 'trained_trans']

merged_df_valid.head(2)

,Original_Language,Original_Poem,English_Poem,untrianed_translation,trained_trans
0,HIndi,"गुलाब की खुशबू में बसी है,\nहरियाली में ढलती स...","In the fragrance of roses, it resides, \nIn t...","The fragrance of a rose,\nis nestled in greene...",The fragrance of the rose\nIs embedded in the ...
1,Assamese,"নদী\nনদী, তুঁতৰ বাণী বহে\nপ্ৰবাহিত শান্তিৰ বাট...","The River\nO River, your voice flows\nAlong th...","River\n\nRiver, silk thread of words flows\nAl...","River\nRiver, the soft voice flows\nIn the pat..."


In [ ]:
merged_df_valid.tail(2)

,Original_Language,Original_Poem,English_Poem,untrianed_translation,trained_trans
473,Kannada,"ನಿನ್ನ ಹಾಡಿನ ಮೆಲುಕು, \nಬಾಳದ ಹಾದಿಯಲಿ, \nಹೃದಯದ ...","The melody of your song, \nIn the journey of ...","Today's song of joy, \nworn path of nostalgia...","The melody of your song, \nIn the path of lif..."
474,Telugu,హీనుఁడెన్ని విద్యలు నేర్చినఁగాని\n\nఘనుఁడుఁగాడ...,"No matter how many subjects one learns,\nA tru...",హీనుడెన్ని విద్యలు నేర్చినగాని\nఘనుడుగాడు హీనజ...,"Even if he may not have studied many arts,\nHe..."


In [ ]:
df_valid = merged_df_valid

In [ ]:
import textwrap
from textwrap import dedent

EOS_TOKEN = tokenizer.eos_token


def create_dpo_prompt(row):
    prompt = dedent(
        f"""### Intructions:
        You are a distinguished expert in Indian languages and literature,
        with a deep understanding of the nuances, themes, symbolism, and visual imagery in poetry.
        You translate poems to english with precision, capturing their cultural context.

        ### Input:
        Translate the given {row["Original_Language"]} poem to English:
        {row["Original_Poem"]}

        Provide only the translation, without any additional comments or explanations

        ### Translation:
        """
    )

    # return tokenizer.apply_chat_template(messages, tokenize=True , add_generation_prompt = True , return_tensors = "pt").to("cuda")
    return prompt
    # return tokenizer.apply_chat_template(messages, tokenize=False , add_generation_prompt = True)


def create_chosen_prompt(data_row):
    messages = data_row["English_Poem"] + EOS_TOKEN
    return messages
    # return tokenizer.apply_chat_template(messages, tokenize=False )
def create_rejected_prompt(data_row):
    messages =  data_row["untrianed_translation"] + EOS_TOKEN
    return messages
    # return tokenizer.apply_chat_template(messages, tokenize=False )

In [ ]:
df_train['prompt'] = df_train.apply(create_dpo_prompt, axis=1)
df_train['chosen'] = df_train.apply(create_chosen_prompt , axis = 1)
df_train['rejected'] = df_train.apply(create_rejected_prompt , axis = 1)

df_valid['prompt'] = df_valid.apply(create_dpo_prompt, axis=1)
df_valid['chosen'] = df_valid.apply(create_chosen_prompt , axis = 1)
df_valid['rejected'] = df_valid.apply(create_rejected_prompt , axis = 1)

df_train.head(2)

,Unnamed: 0,Original_Language,Original_Title,English_Title,Original_Poem,English_Poem,Poet,Translator,token_length,formatted_text,trained_trans,untrianed_translation,prompt,chosen,rejected
0,0,Telugu,Balavanthudu,The Nature of Strength,బలవంతుcడ నాకేమని \nబలువురతో నిగ్రహించి పలుకుట...,What is the strength of the powerful? \nIs it...,Baddena Bhupaludu,NaN,306,<|begin_of_text|><|start_header_id|>system<|en...,"What good is strength, I ask, \nWhen spoken h...",Balaratno naakeemani \nBalaratoho nigraahi ni...,### Intructions:\n You are a distinguis...,What is the strength of the powerful? \nIs it...,Balaratno naakeemani \nBalaratoho nigraahi ni...
1,1,Kannada,Haage Summane,Just Like That\n,"ಹಾಗೆ ಸುಮ್ನೆ ನೆನೆಸಿ, \nಬಾಳದಲಿ ಹಿತವೆನು, \nಹಸುವ...","Just like that, remembering, \nIs there joy i...",G. S. Shivarudrappa\n,NaN,229,<|begin_of_text|><|start_header_id|>system<|en...,"Just remember, \nIn life, joy, \nThe warmth ...","Like the mist of morning dew,\nLike a balm to ...",### Intructions:\n You are a distinguis...,"Just like that, remembering, \nIs there joy i...","Like the mist of morning dew,\nLike a balm to ..."


In [ ]:
df_train["chosen"][0], df_train["rejected"][0]

('What is the strength of the powerful?  \nIs it merely to speak harshly to the weak?  \nA powerful serpent  \nCan still be trapped and killed by tiny ants, oh Sumathi.<|eot_id|>',
 'Balaratno naakeemani  \nBalaratoho nigraahi nigelakuatameela  \nBalaratmayina serpmu  \nChalichima chetjike chavadhaa sumatee.<|eot_id|>')

In [ ]:
df_train.head(2)

,Unnamed: 0,Original_Language,Original_Title,English_Title,Original_Poem,English_Poem,Poet,Translator,token_length,formatted_text,trained_trans,untrianed_translation,prompt,chosen,rejected
0,0,Telugu,Balavanthudu,The Nature of Strength,బలవంతుcడ నాకేమని \nబలువురతో నిగ్రహించి పలుకుట...,What is the strength of the powerful? \nIs it...,Baddena Bhupaludu,NaN,306,<|begin_of_text|><|start_header_id|>system<|en...,"What good is strength, I ask, \nWhen spoken h...",Balaratno naakeemani \nBalaratoho nigraahi ni...,### Intructions:\n You are a distinguis...,What is the strength of the powerful? \nIs it...,Balaratno naakeemani \nBalaratoho nigraahi ni...
1,1,Kannada,Haage Summane,Just Like That\n,"ಹಾಗೆ ಸುಮ್ನೆ ನೆನೆಸಿ, \nಬಾಳದಲಿ ಹಿತವೆನು, \nಹಸುವ...","Just like that, remembering, \nIs there joy i...",G. S. Shivarudrappa\n,NaN,229,<|begin_of_text|><|start_header_id|>system<|en...,"Just remember, \nIn life, joy, \nThe warmth ...","Like the mist of morning dew,\nLike a balm to ...",### Intructions:\n You are a distinguis...,"Just like that, remembering, \nIs there joy i...","Like the mist of morning dew,\nLike a balm to ..."


In [ ]:
df_valid.shape , df_train.shape

((475, 8), (1095, 15))

In [ ]:
df_train_run = df_train[['prompt' , 'chosen' , 'rejected']]
# df_valid_run = df_valid[['prompt' , 'chosen' , 'rejected']]


In [ ]:
df_train_run.to_json("train.json", orient="records", lines=True)
# df_valid_run.to_json("val.json", orient="records", lines=True)

In [ ]:
import json
from datasets import load_dataset

# Load the dataset and access the 'train' split directly
orpo_dataset = load_dataset("json", data_files="train.json", split="train")

# Print the dataset
print(orpo_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1095
})


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj", ],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [ ]:
from trl import ORPOConfig
from unsloth import is_bfloat16_supported
import torch
train_args = ORPOConfig(

        # do_eval = True,
        # evaluation_strategy = "epoch",
        max_length = max_seq_length,
        max_prompt_length = max_seq_length,
        max_completion_length = 512,
        per_device_train_batch_size = 4,
        # per_device_eval_batch_size = 8,
        gradient_accumulation_steps = 4,
        beta = 0.1,
        warmup_ratio = 0.1,
        save_strategy = "epoch",
        logging_steps = 20,
        learning_rate = 5e-5,
        optim = "adamw_8bit",
        lr_scheduler_type = "cosine",
        num_train_epochs = 3,
        # max_steps = 30, # Change to num_train_epochs = 1 for full training runs
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )

In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

response_template = "<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
from trl import ORPOConfig, ORPOTrainer
from unsloth import is_bfloat16_supported

orpo_trainer = ORPOTrainer(
    model = model,
    train_dataset=orpo_dataset,  # Access the 'train' split if using DatasetDict
    # eval_dataset=orpo_dataset["validation"],
    # disable_prompt_loss = True,
    tokenizer = tokenizer,
    args = train_args
)

Map:   0%|          | 0/1095 [00:00<?, ? examples/s]

Map:   0%|          | 0/1095 [00:00<?, ? examples/s]

In [ ]:
orpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,095 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 204
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
20,0.552200,-0.019306,-0.415210,1.000000,0.395904,-4.152101,-0.193057,0.350799,0.373466
40,0.528900,-0.017121,-0.453119,1.000000,0.435997,-4.531185,-0.171212,0.361484,0.388129
60,0.544500,-0.018596,-0.413505,1.000000,0.394910,-4.135053,-0.185958,0.340041,0.356991
80,0.467400,-0.011943,-0.459612,1.000000,0.447669,-4.596118,-0.119430,0.288231,0.314165
100,0.451700,-0.009630,-0.480995,1.000000,0.471366,-4.809954,-0.096297,0.315201,0.350701
120,0.445100,-0.011152,-0.494090,1.000000,0.482938,-4.940902,-0.111517,0.286533,0.335496
140,0.470000,-0.012336,-0.457961,1.000000,0.445626,-4.579610,-0.123355,0.259958,0.290245
160,0.377700,-0.006267,-0.506704,1.000000,0.500437,-5.067039,-0.062672,0.283777,0.332672
180,0.407000,-0.007046,-0.548131,1.000000,0.541085,-5.481311,-0.070460,0.314297,0.367348
200,0.395700,-0.006112,-0.488938,1.000000,0.482826,-4.889380,-0.061124,0.417697,0.464792


TrainOutput(global_step=204, training_loss=0.4637740079094382, metrics={'train_runtime': 1048.8872, 'train_samples_per_second': 3.132, 'train_steps_per_second': 0.194, 'total_flos': 0.0, 'train_loss': 0.4637740079094382, 'epoch': 2.9635036496350367})

In [ ]:
model_path = "outputs/checkpoint-138"
model_path

'/content/outputs/checkpoint-138'

In [ ]:
from huggingface_hub import HfApi

username = "Cherran"
MODEL_NAME = "ORPO_MODEL"
api = HfApi(token=  "hf_token...")

api.create_repo(
    repo_id = f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    repo_id = f"{username}/{MODEL_NAME}",
    folder_path = model_path
)

  0%|          | 0/6 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/171M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Cherran/best_more_epoch_less_performance_orpo_sft_untrained_llama_epoch_2/commit/186493c8effdd6db8880ccc3392bcafc182f99f3', commit_message='Upload folder using huggingface_hub', commit_description='', oid='186493c8effdd6db8880ccc3392bcafc182f99f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Cherran/best_more_epoch_less_performance_orpo_sft_untrained_llama_epoch_2', endpoint='https://huggingface.co', repo_type='model', repo_id='Cherran/best_more_epoch_less_performance_orpo_sft_untrained_llama_epoch_2'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

FastLanguageModel.for_inference(model)
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 512,
    temperature = 1,
    min_p = 0.1,
    return_full_text = False
)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

In [ ]:
import textwrap
from tqdm import tqdm
from textwrap import dedent

def pipe_create_test_prompt(row):
    # print(row["Original_Language"])
    prompt = dedent(
        f"""
        Translate the given {row["Original_Language"]} poem to english:
        {row["Original_Poem"]}

        Provide only the english translation, without any additional comments, explanations and other texts:

        Translation:
        """
    )
    messages = [
        {
            "role": "system",
            "content": "You are a distinguished expert in Indian languages and literature, with a deep understanding of the nuances, themes, symbolism, and visual imagery in poetry. You analyze poems with precision, capturing their cultural context",
            },
        {
            "role": "user",
            "content": prompt,
        }
    ]
    # return tokenizer.apply_chat_template(messages, tokenize=True , add_generation_prompt = True , return_tensors = "pt").to("cuda")
    return tokenizer.apply_chat_template(messages, tokenize=False , add_generation_prompt = True)

def get_inefered_rows( df ) :
  df_ = df.copy()
  df_ = df_.to_dict(orient='records')

  rows = []
  for row in tqdm(df.to_dict(orient = 'records')):
      prompt = pipe_create_test_prompt(row)
      outputs = pipe(prompt)
      row_trained.append( outputs[0]["generated_text"])
      if(len(row_trained)%20 == 0):
        print( outputs[0]["generated_text"] )

  return rows

In [ ]:
import pandas as pd
df_valid = pd.read_excel('/content/gemmaQuadra_mistralTriple_qwenTriple_LlamaDbl_val_data.xlsx')
df_valid.head(1)

,Original_Language,Original_Title,English_Title,Poet,Translator,Original_Poem,English_Poem,token_length,formatted_text,gemma_trained_trans,...,gemma_untrained,gemma_orpo_trained,lowrank_gemma_trained_trans,gemma_dpo_trained,mistral_dpo_trained,qwen_dpo_trained,mistral_ORPO,qwen_orpo_trained,llama_trained,llama_untrianed
0,HIndi,Ghar Ki Yaad,Memory of Home,Kumar Vishwas\n,NaN,"गुलाब की खुशबू में बसी है,\nहरियाली में ढलती स...","In the fragrance of roses, it resides, \nIn t...",232,<|begin_of_text|><|start_header_id|>system<|en...,"In the fragrance of roses,\nThe morning melts ...",...,"The fragrance of roses lingers,\n The m...","In the fragrance of roses,\nMornings fade into...","In the fragrance of roses,\nThe morning melts ...","In the fragrance of roses,\nThe morning melts ...","In the fragrance of roses lies,\nGreen leaves ...","In the fragrance of of roses,\nThe mornings fa...","In the fragrance of the rose,\nLies the freshn...","In the fragrance of roses,\nMorning blends int...",The fragrance of the rose\nIs embedded in the ...,"The fragrance of a rose,\nis nestled in greene..."


In [ ]:
df_valid.columns

Index(['Original_Language', 'Original_Title', 'English_Title', 'Poet',
       'Translator', 'Original_Poem', 'English_Poem', 'token_length',
       'formatted_text', 'gemma_trained_trans', 'mistral_trained_trans',
       'qwen_trained_trans', 'gemma_untrained', 'gemma_orpo_trained',
       'lowrank_gemma_trained_trans', 'gemma_dpo_trained',
       'mistral_dpo_trained', 'qwen_dpo_trained', 'mistral_ORPO',
       'qwen_orpo_trained', 'llama_trained', 'llama_untrianed'],
      dtype='object')

In [ ]:
infered_rows = get_inefered_rows(df_valid)

  4%|▍         | 20/470 [01:10<29:09,  3.89s/it]

The silence of the breeze,  
How deep it lies.  
In every breath,  
A story it hides.


  9%|▊         | 40/470 [02:26<24:52,  3.47s/it]

First Talent
The first talent in my heart,
Like the first ray of the sun
comes to the desolate landscape,
A flower blooms in my mind.


 13%|█▎        | 60/470 [03:43<25:14,  3.69s/it]

...

 94%|█████████▎| 440/470 [27:45<02:04,  4.16s/it]

Listen, Draupadi, take up arms now,  
For Govind will not come this time.  
On your stripping,  
No Krishna will intervene.  

These ashes, dust, these particles of shame,  
Will only be with you, all alone.  
You must now forge the code of law,  
And uphold justice, unbroken.  

In this darkness of battle,  
You will be your own light.  
Power lies now in your hands,  
You will chart your own course.


 98%|█████████▊| 460/470 [28:44<00:29,  2.96s/it]

Proud, like the lion,  
This is the mark of battle,  
In the tricolor lies,  
The story of our freedom.


100%|██████████| 470/470 [29:22<00:00,  3.75s/it]


In [ ]:
%%capture
!pip install unbabel-comet  # First install COMET
!pip install nltk rouge-score sacrebleu

import nltk
import numpy as np
from rouge_score import rouge_scorer
from sacrebleu.metrics import BLEU
from nltk.translate.bleu_score import sentence_bleu
from comet import download_model, load_from_checkpoint
nltk.download('punkt')
nltk.download('wordnet')


from nltk.translate.bleu_score import sentence_bleu
import numpy as np

def calculate_scores(df, true, pred, source ):
    """
    Calculates BLEU, METEOR, ROUGE, and COMET scores for a dataframe.

    Args:
        df: DataFrame containing the text columns
        true: Name of the column containing reference/ground truth text
        pred: Name of the column containing predicted/generated text

    Returns:
        scores: Dictionary with average scores
        all_scored: Dictionary with individual scores for each sample
    """
    # Initialize score lists
    bleu_scores = {'bleu1': [], 'bleu2': [], 'bleu3': [], 'bleu4': []}
    meteor_scores = []
    rouge_scores = []
    comet_scores = []

    # Initialize scorers
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Prepare data for COMET
    comet_data = [{
        'src': row[source],  # Empty source for monolingual evaluation
        'mt': row[pred],
        'ref': row[true]
    } for _, row in df.iterrows()]

    # Load COMET model
    comet_model = load_from_checkpoint(download_model('wmt20-comet-da'))
    # Calculate COMET scores
    comet_scores = comet_model.predict(comet_data, batch_size=8, gpus=1)

    for idx, row in df.iterrows():
        true_text = row[true]
        predicted_text = row[pred]

        # Calculate BLEU scores
        reference = [true_text.split()]
        candidate = predicted_text.split()

        # Calculate different BLEU variants
        weights = [(1, 0, 0, 0), (0.5, 0.5, 0, 0),
                  (0.33, 0.33, 0.33, 0), (0.25, 0.25, 0.25, 0.25)]
        for n, weight in enumerate(weights, 1):
            bleu_scores[f'bleu{n}'].append(
                sentence_bleu(reference, candidate, weights=weight)
            )

        # Calculate METEOR score
        meteor_scores.append(
            nltk.translate.meteor_score.single_meteor_score(
                true_text.split(), predicted_text.split()
            )
        )

        # Calculate ROUGE scores
        rouge_scores.append(rouge_scorer_obj.score(true_text, predicted_text))

    # Compile all scores
    all_scored = {
        'bleu1': bleu_scores['bleu1'],
        'bleu2': bleu_scores['bleu2'],
        'bleu3': bleu_scores['bleu3'],
        'bleu4': bleu_scores['bleu4'],
        'meteor': meteor_scores,
        'rouge1': [score['rouge1'].fmeasure for score in rouge_scores],
        'rouge2': [score['rouge2'].fmeasure for score in rouge_scores],
        'rougeL': [score['rougeL'].fmeasure for score in rouge_scores],
        'comet': comet_scores['scores']  # Add COMET scores
    }

    # Calculate average scores
    scores = {
        key: round(np.array(value).mean(), 4)
        for key, value in all_scored.items()
    }

    return scores, all_scored

In [ ]:
scores_orpo, _ = calculate_scores(df_valid , 'English_Poem' , 'llama_orpo_trained' , 'Original_Poem')
scores_orpo

wmt20-comet-da.tar.gz: 1.79GB [00:52, 34.2MB/s]                            
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/unbabel_comet/wmt20-comet-da/checkpoints/model.ckpt`


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 59/59 [00:06<00:00,  9.38it/s]
/us

{'bleu1': 0.5038,
 'bleu2': 0.3802,
 'bleu3': 0.296,
 'bleu4': 0.2277,
 'meteor': 0.5094,
 'rouge1': 0.6421,
 'rouge2': 0.3803,
 'rougeL': 0.5791,
 'comet': 0.3168}

In [ ]:
df_valid.columns

Index(['Original_Language', 'Original_Title', 'English_Title', 'Poet',
       'Translator', 'Original_Poem', 'English_Poem', 'token_length',
       'formatted_text', 'gemma_trained_trans', 'mistral_trained_trans',
       'qwen_trained_trans', 'gemma_untrained', 'gemma_orpo_trained',
       'lowrank_gemma_trained_trans', 'gemma_dpo_trained',
       'mistral_dpo_trained', 'qwen_dpo_trained', 'mistral_ORPO',
       'qwen_orpo_trained', 'llama_trained', 'llama_untrianed',
       'llama_orpo_trained'],
      dtype='object')

In [ ]:
df_valid.to_excel('gemmaQuadra_mistralTriple_qwenTriple_LlamaTriple_val_data.xlsx' , index = False)

## Inference

In [ ]:
import pandas as pd

df = pd.read_excel("/content/multi_poems_trained+untrained.xlsx")
df.head()

,Unnamed: 0,Original_Language,Original_Title,English_Title,Original_Poem,English_Poem,Poet,Translator,token_length,formatted_text,trained_trans,untrianed_translation
0,0,Bengali,Chitto Jetha Bhayshunyo,Where the Mind is Without Fear,"চিত্ত যেথা ভয়শূন্য, উচ্চ যেথা শির,\nজ্ঞান যেথ...",Where the mind is without fear and the head is...,Rabindranath Tagore,Rabindranath Tagore,796,<|begin_of_text|><|start_header_id|>system<|en...,"Wherever fear is absent, where the mind is hig...","In the depth where fear disappears, in..."
1,1,Bengali,Banalata Sen,Banalata Sen,"শ্রাবণী নিশার গভীর নীরবতা,\nশালিকের বীণায়, একট...","In the deep silence of the rainy night,\nIn th...",Jibanananda Das,Anne Walmsley,1185,<|begin_of_text|><|start_header_id|>system<|en...,"In the deep silence of the rainy night,\nIn th...","In the darkness of the night of Shravan,\nThe ..."
2,2,Bengali,Aaj Nover Sukh,The Happiness of Today,"আজ নবের সুখ,\nবাতাসে বহে।\nমনেতে আশা,\nনব উদয়ে...","Today is the happiness of a new dawn,\nBlowing...",Kazi Nazrul Islam,M. R. Choudhury,781,<|begin_of_text|><|start_header_id|>system<|en...,"Today, a new joy, \nSpreads in the air. \nHo...","Today brings me joy,\nIt flows on the breeze.\..."
3,3,Bengali,O Amar Desher Mati,O Soil of My Country,"ও আমার দেশের মাটি,\nতোমার কল্যাণে চাহি,\nপ্রাণ...","O soil of my country,\nIn your welfare, I seek...",Kazi Nazrul Islam,Keki N. Daruwalla,709,<|begin_of_text|><|start_header_id|>system<|en...,"O soil of my land,\nIn your grace, I seek,\nTo...","Oh my country's earth, \n\nIn the sake of thy ..."
4,4,Bengali,Ekla Cholo Re,Walk Alone,"একলা চলো রে,\nযদি কেউ আসে না।\nযদি আমি পথ ভুলে...",Walk alone.\nIf no one comes along.\nIf I lose...,Rabindranath Tagore,Rabindranath Tagore,594,<|begin_of_text|><|start_header_id|>system<|en...,"Walk alone,\nIf no one comes.\nIf I lose my wa...","Walk alone, O my love,\n\nIf someone doesn't c..."


In [ ]:
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Cherran/best_orpo_sft_untrained_english_translate_model_llama3.1_inst8b_epoch_3",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.12.9: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Unsloth 2024.12.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from transformers import pipeline

FastLanguageModel.for_inference(model)
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 512,
    temperature = 1,
    min_p = 0.1,
    return_full_text = False
)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

In [ ]:
import textwrap
from tqdm import tqdm
from textwrap import dedent

def pipe_create_test_prompt(row):
    # print(row["Original_Language"])
    prompt = dedent(
        f"""
        Translate the given {row["Original_Language"]} poem to english:
        {row["Original_Poem"]}

        Provide only the english translation, without any additional comments, explanations and other texts:

        Translation:
        """
    )
    messages = [
        {
            "role": "system",
            "content": "You are a distinguished expert in Indian languages and literature, with a deep understanding of the nuances, themes, symbolism, and visual imagery in poetry. You analyze poems with precision, capturing their cultural context",
            },
        {
            "role": "user",
            "content": prompt,
        }
    ]
    # return tokenizer.apply_chat_template(messages, tokenize=True , add_generation_prompt = True , return_tensors = "pt").to("cuda")
    return tokenizer.apply_chat_template(messages, tokenize=False , add_generation_prompt = True)

def get_inefered_rows( df ) :
  df_ = df.copy()
  df_ = df_.to_dict(orient='records')

  row_trained = []
  for row in tqdm(df.to_dict(orient = 'records')):
      prompt = pipe_create_test_prompt(row)
      outputs = pipe(prompt)
      row_trained.append( outputs[0]["generated_text"])
      if(len(row_trained)%20 == 0):
        print( outputs[0]["generated_text"] )

  return row_trained

In [ ]:
infered_rows = get_inefered_rows(df)

  1%|▏         | 20/1569 [01:37<1:04:52,  2.51s/it]

Without You
Without you, my days pass
Alone, without light.
Without you, in the path of life,
There is an emptiness.


  3%|▎         | 40/1569 [02:17<51:51,  2.03s/it]

Welcome, you youth,
Welcome, you youth, to my heart,
New, dreams, woven,
By life’s melody,
Songs of dreams.


  4%|▍         | 60/1569 [03:30<2:51:49,  6.83s/it]

...

 98%|█████████▊| 1540/1569 [1:37:39<00:54,  1.87s/it]

The sun of dawn rose,  
Spirit filled with zeal.  
With new hopes in tow,  
Every path revealed itself.


 99%|█████████▉| 1560/1569 [1:38:17<00:15,  1.67s/it]

My dhol, earth’s color in shade.
The innocent dance, singing songs of love.


100%|██████████| 1569/1569 [1:38:36<00:00,  3.77s/it]


In [ ]:
df["orpo_trained"] = infered_rows

In [ ]:
df.head(2)

,Unnamed: 0,Original_Language,Original_Title,English_Title,Original_Poem,English_Poem,Poet,Translator,token_length,formatted_text,trained_trans,untrianed_translation,orpo_trained
0,0,Bengali,Chitto Jetha Bhayshunyo,Where the Mind is Without Fear,"চিত্ত যেথা ভয়শূন্য, উচ্চ যেথা শির,\nজ্ঞান যেথ...",Where the mind is without fear and the head is...,Rabindranath Tagore,Rabindranath Tagore,796,<|begin_of_text|><|start_header_id|>system<|en...,"Wherever fear is absent, where the mind is hig...","In the depth where fear disappears, in...","Wherever fear recedes, there high is the head;..."
1,1,Bengali,Banalata Sen,Banalata Sen,"শ্রাবণী নিশার গভীর নীরবতা,\nশালিকের বীণায়, একট...","In the deep silence of the rainy night,\nIn th...",Jibanananda Das,Anne Walmsley,1185,<|begin_of_text|><|start_header_id|>system<|en...,"In the deep silence of the rainy night,\nIn th...","In the darkness of the night of Shravan,\nThe ...","The deep silence of the rainy night,\nIn the t..."


In [ ]:
df.to_excel("multi_poems_orpo_trained+untrained.xlsx", index=False)